In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score, precision_score, mean_squared_error, roc_auc_score
from sklearn.tree import DecisionTreeClassifier, export_text
from sklearn.svm import SVC

In [ ]:
X_train=pd.read_pickle('Datasets/X_train.plk')
X_test=pd.read_pickle('Datasets/X_test.plk')
y_train=pd.read_pickle('Datasets/y_train.plk')
y_test=pd.read_pickle('Datasets/y_test.plk')

print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

Three algorithms were tested, to know if they are suitable for capturing the differences between the last plate apperances for the pitchers, and a "normal" plate appearance.
They are simple algorithms, whose decision boundaries may be calculated by hand.

Different parameters were randomly tested and all of the variables were used. Here, the "best" results are shown.

# Training the model

### Logistic Regression

In [ ]:
lr=LogisticRegression(random_state=42, penalty='none', max_iter=10000)
lr.fit(X_train, y_train)

In [ ]:
y_pred=lr.predict(X_test)

pd.crosstab(y_test, y_pred, margins=True)

In [ ]:
print( accuracy_score(y_test, (y_pred>0.5)+0 ))
print( precision_score(y_test, (y_pred>0.5)+0 ))
print( roc_auc_score(y_test, (y_pred>0.5)+0 ))
print( f1_score(y_test, (y_pred>0.5)+0 ))

### SVM 

In [ ]:
svm=SVC(kernel='linear', C=0.5)
svm.fit(X_train, y_train)

In [ ]:
y_pred2=svm.predict(X_test)

pd.crosstab(y_test, np.round(y_pred2,0), margins=True)

In [ ]:
print( accuracy_score(y_test, (y_pred2>0.5)+0 ))
print( precision_score(y_test, (y_pred2>0.5)+0 ))
print( roc_auc_score(y_test, (y_pred2>0.5)+0 ))
print( f1_score(y_test, (y_pred2>0.5)+0 ))

### Tree

In [ ]:
train=pd.read_csv('train.csv')
test=pd.read_csv('test.csv')

X_train=train.iloc[:,0:67]
y_train=train.iloc[:,69]

#X_test=test.iloc[:,0:67]
#y_test=test.iloc[:,68]


In [ ]:
tree=DecisionTreeClassifier(max_depth=3, criterion='gini', random_state=42)
tree.fit(X_train, y_train)

In [ ]:
y_pred3=tree.predict(X_test[X_train.columns.tolist()])

pd.crosstab(y_test, (y_pred3>0.5)+0, margins=True)

In [ ]:
print( accuracy_score(y_test, (y_pred3>0.5)+0 ))
print( precision_score(y_test, (y_pred3>0.5)+0 ))
print( roc_auc_score(y_test, (y_pred3>0.5)+0 ))
print( f1_score(y_test, (y_pred3>0.5)+0 ))

In [ ]:
X_train.columns[tree.feature_importances_>0].tolist()

The three algorithms performed similar. In particular, the SVM and the Logistic Regression had a similar prediction on the testing set. That was expected, as they both aim to generate the best “hyperplane” that separates the two classes. However, the logistic regression runs noticeably faster, and has a more concise formula to determine the decision boundary. Therefore, it will be chosen over the SVM with a linear kernel.

On the other hand, the Decision Tree performed slightly better on most of the metrics while using less variables. That is because the maximum depth was set at four. Thus, the Decision Tree will be used as a base model, while we attempt to have a better model by using a Logistic Regression. 


In [ ]:
print( export_text(tree,  feature_names=X_train.columns.tolist()) )